Attempting to predict the variables that determine if blue side wins: https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min.

The goal of analysing this dataset is to inform high-elo League of Legends players the key metrics to focus on in the first 10 minutes ie. the 'Laning Phase' (Ferrari, 2013) in order to maximise your chances of winning the game overall.

The goal is also to use two statistical methods on a single dataset then potentially use it to train a logistic regression model: 

    1. Factor Analysis

    2. Principal Component Analysis

Both of these methods (in my opinion) aren't well documented, so hopefully this will be relevant for some who were as lost as I was! Throughout this notebook, I've placed some references and consolidated pieces of their methodologies here.

    Please note that this data captures the first 10 minutes of a game. In a 30-40 minute game, a lot of 'comebacks' can happen, so our final accuracy will not be 'overly' magnificent. 
    
    From other individuals who have analysed this dataset, a 70-75% final accuracy seems to be the concensus. 
    
    Therefore, having a strong advantage in the 1st 10 minutes of the game would mean your team had a 70-75% chance of winning the game overall.

First, load packages.

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn.datasets
from matplotlib import pyplot as plt
from factor_analyzer import FactorAnalyzer

Next, load dataset.

In [ ]:
lol = pd.read_csv("../input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv")
lol.info()
pd.options.display.max_columns = None
lol.head(20)

In [ ]:
#We'll drop missing values rows. No need to as there are 9879 non-null vlaues for each column for the total 9879 entries. Still good practice:
lol.dropna(inplace=True)

In [ ]:
#setting up our new DataFrame for Factor Analysis
df_clean = lol.copy()
df_clean.head(10)

Cleaning up our dataframe:

In [ ]:
#We're going to eliminate variables with reasons why:
#1. gameId is an obvious candidate to eliminate since the unique ID and how it varies plays no role. 
df_clean = df_clean.drop('gameId',axis=1)
#2. For the purpose of FA, we can drop blueWins (although we will retain it for regression later).
df_clean = df_clean.drop('blueWins',axis=1)
y = lol['blueWins']
#3. We can eliminate elite monster kills as it can be derived from dragon kills + herald kills.
df_clean = df_clean.drop('blueEliteMonsters',axis=1)
df_clean = df_clean.drop('redEliteMonsters',axis=1)
#4. blueFirstBlood has a perfect negative correlation with redFirstBlood, so we'll drop one of them.
df_clean = df_clean.drop('redFirstBlood',axis=1)
#5. blueGoldDiff/blueExperienceDiff are calculated variables too -> blueTotalGold - redTotalGold & blueTotalExperience - redTotalExperience.
df_clean = df_clean.drop('blueGoldDiff',axis=1)
df_clean = df_clean.drop('redGoldDiff',axis=1)
df_clean = df_clean.drop('blueExperienceDiff',axis=1)
df_clean = df_clean.drop('redExperienceDiff',axis=1)
#6. Because the dataset's context is game stats until the 10 minute mark, GPM and XPM are literally just total gold/exp divided by 10. We can drop them.
df_clean = df_clean.drop('blueCSPerMin',axis=1)
df_clean = df_clean.drop('redCSPerMin',axis=1)
df_clean = df_clean.drop('blueGoldPerMin',axis=1)
df_clean = df_clean.drop('redGoldPerMin',axis=1)
#7. blueKills has a perfect negative correlation with redDeaths & blueDeaths has a perfect negative correlation with blueDeaths.
df_clean = df_clean.drop('redKills',axis=1)
df_clean = df_clean.drop('redDeaths',axis=1)
#8. While not perfectly correlated, avglevel and totalExperience are highly correlated. We will drop one of them to avoid colinearity. Because leveling up becomes incrementally more difficult as you grow in levels, the average level is not the best indication of experience. Better to keep total experience as a better indication.
df_clean = df_clean.drop('blueAvgLevel',axis=1)
df_clean = df_clean.drop('redAvgLevel',axis=1)

In [ ]:
df_clean.info()

Observing the correlation matrix: (Note that 23 dimensions is still extremely high, and we are looking to reduce this by a substantial amount while minimising the loss of information by removing variables)

In [ ]:
fig, ax = plt.subplots(figsize=(25,25))
ax = sns.heatmap(df_clean.corr(), annot=True, ax=ax, cmap="YlGnBu", linewidths = .5); ax.set_title("Correlation between LoL variables", fontsize = 50)
plt.show()
#we can see there is still high colinearity between some variables, but for now we've eliminated most of the variables which would've had correlations > 0.9 with our retained variables. We'll continue to proceed with what we have, but we'll look to eliminate a lot more variables!

Quickly check the correlation between our chosen variables and 'blueWins':

In [ ]:
win = pd.DataFrame({'blueWins':df_clean.corrwith(y,axis=0)})
fig, ax = plt.subplots()
x_labels = ['blueWins']
y_labels = df_clean.columns.tolist()
sns.set(font_scale=0.5)
plt.title('Correlation with winning')
sns.heatmap(win,cmap="YlGnBu", xticklabels = x_labels, yticklabels = y_labels, center=0, square=True, linewidths=0.05,cbar_kws={"shrink": 0.5}, annot = True, annot_kws={"fontsize":1})

We could naively conclude (simply from the correlation matrix analysis) that the more kills, gold and experience Blue team has, the more likely they are going to win. Similarly, the more kills, gold and experience Red team has, the more likely Blue Team will lose. But we want more than just that!

<u><b>Factor Analysis</b></u>

The first method we will try out is Exploratory Factor Analysis (FA). FA will determine a 'cluster' of factors that are significant to the dataset. FA can be difficult compared to Principal Component Analysis (PCA) due to the ambiguity and 'art' of FA.

Generally as a rule of thumb, we run FA if we assume or wish to test a theoretical model of latent factors causing observed variables.

    What I mean by latent factors is similar to causation - winning your lane is a latent cause of more gold, exp, CS, kills and assists for example.

<b>Factor Analysis consists of:</b>

    1. Kaiser Criterion/Scree Plot of eigenvalues to determine number of Factors

    2. Factor Loadings of variables by Factor Analysis Rotation

    3. Communality-Uniqueness Analysis

We're going to perform a <i>'psuedo'</i> Factor Analysis. This will not be the Final Factor Analysis because of the very high colinearity of variables. We will decide on the appropriate method and rotation, run the FA, remove any items with low communalities and re-run (Centre of Academic Success, 2017).

In [ ]:
#Calculating eigenvalues and seeing which ones are greater than 1:
fa = FactorAnalyzer()
fa.fit(df_clean)
ev,v = fa.get_eigenvalues()
count = sum(1 for i in ev if i > 1)
print(count)
big_evs = sum(i for i in ev if i > 1)
total_evs = sum(ev)
print(big_evs)
print(float(big_evs/total_evs))

An eigenvalue > 1 means that the factor explains more variance than a unique variable. Total eigenvalues > 1 = 9.

The total of eigenvalues = 16.43, with the 4 eigenvalues > 1 accounting for 71.44% of the total variance.

In [ ]:
#plotting scree-plot:
plt.scatter(range(1,df_clean.shape[1]+1),ev)
plt.plot(range(1,df_clean.shape[1]+1),ev) 
plt.title('Scree Plot')
plt.xlabel('Factors')
plt.ylabel('Eigenvalue')
plt.axhline(1.0, color = 'black') #visualise which eigenvalues are above and below 1
plt.grid(b=True, which='major', color='#666666', linestyle='-')
# Show the minor grid lines with very faint and almost transparent grey lines
plt.minorticks_on()
plt.grid(b=True, which='minor', color='#999999', linestyle='-.')
plt.show()
#visualisation of the 9 eigenvalues > 1.

What does the above mean?

Our naive Factor Analysis Approach involves us making a rough estimate on the number of Factors required in FA. The rough criteria is that the number of eigenvalues > 1 is the number of Factors we keep (Kaiser Criterion). However this criteria is <b>highly</b> disputed and shouldn't be gospel (Hayton, Allen & Scarpello, 2004).

For now, we'll perform FA on 9 factors.

<u>FA Rotation selection:</u>

Generally, varimax rotation is chosen to maximise sum(variance of squared loadings) and ensure factors created are orthogonal (uncorrelated). However, imposing orthogonality on highly correlated factors is more 'unnatural'. Therefore, we will do an oblique rotation. The drawback of doing oblimin rotation is diminished interpretability of factors due to higher eigenvalues (IBM, 2014).).

In [ ]:
#Performing Factor Analysis:
fa2 = FactorAnalyzer(count, rotation="varimax")
fa2.fit(df_clean)
x_labels = ['Factor ' + str(i) for i in range(1,count+1)]
y_labels = df_clean.columns.tolist()
sns.set(font_scale=0.5)
plt.title('Loading Factors - ' + str(count))
load = sns.heatmap(fa2.loadings_,cmap="coolwarm", xticklabels = x_labels, yticklabels = y_labels, center=0, square=True, linewidths=.2,cbar_kws={"shrink": 0.5}, annot = True, annot_kws={"fontsize":1})

Ideally, we would like to see that each variable is highly correlated with only 1 factor. We can see from above that the wards placed/destroyed and first blood do not have a high enough factor loading for any of the 9 Factors (cut-off being 0.5). Furthermore, Factors 8 & 9 do not have a high enough loading for any variables.

<u>Communality-Uniqueness Analysis:</u>

Uniqueness is the variance that is 'unique' to the variable (not shared), while Communalities is the variance that is common to the variable (shared). Uniqueness + Communalities = 1.

Low Uniqueness/High Communality = Good. High Uniqueness/Low Communality = bad and should be under consideration for elimination. (Princeton University, 2020)

In [ ]:
var_check = np.vstack((fa2.get_communalities(), fa2.get_uniquenesses(),np.array(fa2.get_communalities() + fa2.get_uniquenesses()))).tolist()
y_labels = ['Communality','Uniqueness', 'Total Variance']
x_labels = df_clean.columns.tolist()
sns.set(font_scale=0.5)
plt.title('Communality-Uniqueness of Variables')
load = sns.heatmap(var_check,cmap="RdBu", xticklabels = x_labels, yticklabels = y_labels, center=0, square=True, linewidths=.2,cbar_kws={"shrink": 0.5}, annot = True, annot_kws={"fontsize":1})

The very low Communality/high Uniqueness of variables strongly suggests the removal and the analysis be rerun (cut-off being 0.5). These variables for consideration include:

    1. Wards placed/destroyed for blue and red side (0.98, 0.79, 0.97, 0.88 Uniqueness)

    2. Who got first blood (0.84 Uniqueness)

    3. How many Heralds (0.66, 0.73 Uniqueness)

    4. Towers destroyed (0.73, 0.7 Uniqueness)

    5. Jungle minions destroyed (0.65, 0.65 Uniqueness)

There is enough justification to remove the mentioned variables:

    1. Low correlations in correlation matrix and with blueWins.

    2. None of the 9 proposed factors have high enough loadings for these mentioned variables above.

    3. Very low Communality

<b>We've now finished our 'psuedo' Factor Analysis. From the above, we will look to drop the mentioned variables, and we will perform a proper Factor Analysis after this:</b>

<b>Factor Analysis part 2:</b>

In [ ]:
#dropping mentioned variables:
cols = ['blueWardsPlaced','blueWardsDestroyed','redWardsPlaced','redWardsDestroyed','blueFirstBlood', 'blueHeralds', 'redHeralds', 'blueTowersDestroyed', 'redTowersDestroyed', 'blueTotalJungleMinionsKilled', 'redTotalJungleMinionsKilled']
df_fa = df_clean.drop(cols,axis=1)

In [ ]:
df_fa.head(10)

Observing the correlation matrix with reduced variables:

In [ ]:
#Time to observe the correlation matrix of our dataset:
fig, ax = plt.subplots(figsize=(25,25))
ax = sns.heatmap(df_fa.corr(), annot=True, ax=ax, cmap="YlGnBu", linewidths = .5); ax.set_title("Correlation between LoL variables v2", fontsize = 50)
plt.show()
#We can see there is still high colinearity between some variables, but from the initial Factor Analysis, I can with good conscience say these variables have high communality and are relevant.

In an official and thorough Factor Analysis, we should perform a few tests to validate the justification in performing FA. We will perform two tests on our dataset (Hadi, Abdullah & Sentosa, 2016):

1. <b>Barlett's Test of Sphericity</b> - Checks if our correlation matrix is statistically similar to the Identity matrix. In other words, do our variables essentially have 0 relations with each other or not?

        We want to see a p-value < 0.05 to justify the suitability of FA.

2. <b>Kaiser-Meyer-Olkin (KMO) Test</b> - Checks if the inter-correlations between our variables are strong enough. Similar to Bartlett's test in checking if our variable correlations are sufficient.

        We want to see a KMO measure > 0.6 to justify the suitability of FA.

Performing Barlett's Test of Sphericity:

In [ ]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
calculate_bartlett_sphericity(df_fa)

    In this Bartlett's test, the p-value is 0. This means the test was statistically significant and the observed correlation matrix is not an identity matrix.

Performing the KMO Test - a measure of sampling adequacy (of the strength of inter-correlations among variables) which ranges from 0 to 1:

In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(df_fa)
kmo_model

    In this KMO test, the measure is 0.74. This value is considered good, which justifies suitability of factor analysis for the dataset.

Manually performing feature scaling before running the FA by ensuring our columns have an average mean of 0 and standard deviation of 1:

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_fas = sc.fit_transform(df_fa)

In [ ]:
#Calculating eigenvalues and seeing which ones are greater than 1:
fa = FactorAnalyzer()
fa.fit(df_fas)
ev,v = fa.get_eigenvalues()
count = sum(1 for i in ev if i > 1)
print(count)
big_evs = sum(i for i in ev if i > 1)
total_evs = sum(ev)
print(big_evs)
print(float(big_evs/total_evs))


An eigenvalue > 1 means that the factor explains more variance than a unique variable. Total eigenvalues > 1 = 4.

The total of eigenvalues = 9.76, with the 4 eigenvalues > 1 accounting for 81.36% of the total variance.

In [ ]:
df_fas.shape

In [ ]:
#plotting scree-plot:
plt.scatter(range(1,df_fas.shape[1]+1),ev)
plt.plot(range(1,df_fas.shape[1]+1),ev) 
plt.title('Scree Plot v2')
plt.xlabel('Factors')
plt.ylabel('Eigenvalue')
plt.axhline(1.0, color = 'black') #visualise which eigenvalues are above and below 1
plt.grid(b=True, which='major', color='#666666', linestyle='-')
# Show the minor grid lines with very faint and almost transparent grey lines
plt.minorticks_on()
plt.grid(b=True, which='minor', color='#999999', linestyle='-.')
plt.show()
#visualisation of the 4 eigenvalues > 1.

As mentioned before, we shouldn't rely only on the Kaiser Criterion. There are a lot of methods in retaining factors but there are three of them to utilise (Hayton, Allen & Scarpello, 2004):

    1. Kaiser criterion - if a factor's eigenvalues is above 1.0, we should retain that factor.

    2. Scree plot - when is there a substantial decline in the magnitude of the eigenvalues?

    3. Parallel analysis - given the eigenvalue of the sample and eigenvalue of synthetic data with exact dimensionality of sample, count(the no. of eigenvalues of sample > no. of eigenvalues from synthetic) = retained number of factors.

Using the Kaiser Criterion alone is not good practice as overrelying on it can result in overestimating/underestimating the number of factors. Normally selecting 4 factors would suffice, but the Scree Plot suggests the 5th factor being close to 1 and not being the 'drop-off' point (significant drop-off at 6th factor, where 6th factor onwards plateaus) means there's an argument to actually keep 5 factors. (Li, Yang & Liu, 2018).

**Parallel Analysis:**

Although multiple sources put Parallel Analysis as the most promising method over Kaiser criterion/Scree plot for the number of factors to retain, it could be problematic over 500 subjects (Myzziah, 2021) as well as underestimate the number of factors to retain when the first eigenvalue is large (Beauducel, 2001).

Parallel Analysis is still shown below and still suggests maintaining 4 factors, however we'll still continue keeping the 5 factors.

In [ ]:
def _HornParallelAnalysis(data, K=10, printEigenvalues=False):
# Create a random matrix to match the dataset
    n,m = data.shape
    # Set the factor analysis parameters
    fa = FactorAnalyzer(n_factors=1, method='minres', rotation=None, use_smc=True)
    # Create arrays to store the values
    sumComponentEigens = np.empty(m)
    sumFactorEigens = np.empty(m)
    # Run the fit 'K' times over a random matrix
    for runNum in range(0, K):
        fa.fit(np.random.normal(size=(n, m)))
        sumComponentEigens = sumComponentEigens + fa.get_eigenvalues()[0]
        sumFactorEigens = sumFactorEigens + fa.get_eigenvalues()[1]
    # Average over the number of runs
    avgComponentEigens = sumComponentEigens / K
    avgFactorEigens = sumFactorEigens / K

    # Get the eigenvalues for the fit on supplied data
    fa.fit(data)
    dataEv = fa.get_eigenvalues()
    # Set up a scree plot
    plt.figure(figsize=(8, 6))


    # Print results
    if printEigenvalues:
        print('Principal component eigenvalues for random matrix:\n', avgComponentEigens)
        print('Factor eigenvalues for random matrix:\n', avgFactorEigens)
        print('Principal component eigenvalues for data:\n', dataEv[0])
        print('Factor eigenvalues for data:\n', dataEv[1])
    # Find the suggested stopping points
    suggestedFactors = sum((dataEv[1] - avgFactorEigens) > 0)
    suggestedComponents = sum((dataEv[0] - avgComponentEigens) > 0)
    print('Parallel analysis suggests that the number of factors = ', suggestedFactors , ' and the number of components = ', suggestedComponents)

    # Plot the eigenvalues against the number of variables

    # Line for eigenvalue 1
    plt.plot([0, m+1], [1, 1], 'k--', alpha=0.3)
    # For the random data - Components
    plt.plot(range(1, m+1), avgComponentEigens, 'b', label='PC - random', alpha=0.4)
    # For the Data - Components
    plt.scatter(range(1, m+1), dataEv[0], c='b', marker='o')
    plt.plot(range(1, m+1), dataEv[0], 'b', label='PC - data')
    # For the random data - Factors
    plt.plot(range(1, m+1), avgFactorEigens, 'g', label='FA - random', alpha=0.4)
    # For the Data - Factors
    plt.scatter(range(1, m+1), dataEv[1], c='g', marker='o')
    plt.plot(range(1, m+1), dataEv[1], 'g', label='FA - data')
    plt.title('Parallel Analysis Scree Plots', {'fontsize': 20})
    plt.xlabel('Factors/Components', {'fontsize': 15})
    plt.xticks(ticks=range(1, m+1), labels=range(1, m+1))
    plt.ylabel('Eigenvalue', {'fontsize': 15})
    plt.legend()
    plt.show();
_HornParallelAnalysis(df_fas)

**Performing Factor Analysis and choosing rotation method:**

Because we have variables that have high loadings in factors (blueKills, blueDeaths, blueAssists, blueTotalGold, redAssists, redDragons, RedTotalGold) as well as variables that bleed into other factors (blueTotalExperience, redTotalExperience), we use the 'equamax' rotation method that minimizes both (IBM, 2014).

In [ ]:
#Performing Factor Analysis:
fa2 = FactorAnalyzer(count+1, rotation="equamax")
fa2.fit(df_fas)
x_labels = ['Factor ' + str(i) for i in range(1,count+2)]
y_labels = df_fa.columns.tolist()
sns.set(font_scale=0.5)
plt.figure(figsize=(20,20))
plt.title('Loading Factors - ' + str(count+1) + ' v2')
load = sns.heatmap(fa2.loadings_,cmap="coolwarm", xticklabels = x_labels, yticklabels = y_labels, center=0, square=True, linewidths=.2,cbar_kws={"shrink": 0.5}, annot = True, annot_kws={"fontsize":1})

From the above information, we can interpret the 5 Factors as so:

    1. How many kills has Blue side had on Red?

    2. How many kills has Red side had on Blue?

    3. How many dragons has Blue side killed compared to Red side?

    4. How many last hits does Blue side have?

    5. How many last hits does Red side have?

In [ ]:
var_check = np.vstack((fa2.get_communalities(), fa2.get_uniquenesses(),np.array(fa2.get_communalities() + fa2.get_uniquenesses()))).tolist()
y_labels = ['Communality','Uniqueness', 'Total Variance']
x_labels = df_fa.columns.tolist()
sns.set(font_scale=0.5)
plt.title('Communality-Uniqueness of Variables')
load = sns.heatmap(var_check,cmap="RdBu", xticklabels = x_labels, yticklabels = y_labels, center=0, square=True, linewidths=.2,cbar_kws={"shrink": 0.5}, annot = True, annot_kws={"fontsize":1})

When communalities are all high, it indicates that the extracted components represent the Factors well. In our example, most communalities are very high, with the lowest communality in blueDragons = 0.46.

In [ ]:
#Transform the data with our 5 new Factors:
transformed_df_fa = pd.DataFrame(fa2.transform(df_fas), columns = ['F1','F2','F3','F4','F5'])
transformed_df_fa.head()

There is one last assessment to perform called Cronbach's Alpha. It will measure how interanlly consistent our factors are in relation to how they are as a group. Our Factors will be acceptable when our Cronbach's Alpha is > 0.7 (Hilsdorf, 2020).

In [ ]:
def cronbach_alpha(df):
    # 1. Transform the df into a correlation matrix
    df_corr = df.corr()
    
    # 2.1 Calculate N
    # The number of variables equals the number of columns in the df
    N = df.shape[1]
    
    # 2.2 Calculate R
    # For this, we'll loop through the columns and append every
    # relevant correlation to an array calles "r_s". Then, we'll
    # calculate the mean of "r_s"
    rs = np.array([])
    for i, col in enumerate(df_corr.columns):
        sum_ = df_corr[col][i+1:].values
        rs = np.append(sum_, rs)
    mean_r = np.mean(rs)
    
   # 3. Use the formula to calculate Cronbach's Alpha 
    cronbach_alpha = (N * mean_r) / (1 + (N - 1) * mean_r)
    return cronbach_alpha

Firstly, any variables that are going in an opposite direction need to be reversed for the test:

In [ ]:
df_fa['blueDragonsReversed'] = df_fa['blueDragons'].multiply(-1)
df_fa.head()

Finally, return the cronbach alphas:

In [ ]:
print(cronbach_alpha(df_fa[['blueKills','blueAssists','blueTotalGold']]))
print(cronbach_alpha(df_fa[['blueDeaths','redAssists','redTotalGold','redTotalExperience']]))
print(cronbach_alpha(df_fa[['blueDragonsReversed','redDragons']]))
print(cronbach_alpha(df_fa[['blueTotalExperience','blueTotalMinionsKilled']]))
print(cronbach_alpha(df_fa[['redTotalExperience','redTotalMinionsKilled']]))

Looks like our cronbach alphas for our 5 factors is looking good! Therefore our factors are deemed acceptable.

<b>Performing Logistic Regression after Factor Analysis:</b>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

Splitting our training and test data:

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(transformed_df_fa,y,test_size=0.2,random_state=1)
final_LR=LogisticRegression()

In [ ]:
final_LR.fit(x_train,y_train)

Assessing the Accuracy of our model:

In [ ]:
result_LR=final_LR.predict(x_test)
print(accuracy_score(result_LR,y_test))

That means with the first 10 minutes of League data, we're able to predict the final outcome with a 74.04% accuracy.

Returning the confusion matrix (by row) of True Positives, False Negatives, False Positives & True Negatives:

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, result_LR)
print(confusion_matrix)

Summarizing regression results:

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,transformed_df_fa)
result=logit_model.fit()
print(result.summary2())

<u><b>Principal Component Analysis</b></u>

Defintion: <i>PCA can be defined as the orthogonal projection of the data to a lower dimensional linear space (principal subspace), while minimizing the mean squared distance between datapoints and the orthogonal projection while maximising the spread throughout the orthogonal projection (variance).</i> Extremely great writeup explaining PCA here: https://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues

How is Principal Component Analysis (PCA) different from FA?

1. Both methods are looking to make an approximation of a given covariance matrix. The fundamental mathematical difference between the two methods is this (Bishop, 2014):

   <b>PCA: C ≈ WW<sup>⊤</sup></b>

   <b>FA: C ≈ WW<sup>⊤</sup> + Ψ</b>

    Where C represents the covariance matrix of our dataset, W being a matrix of our data after deciding how many components to keep (like how we retained <i>'k'</i> factors in    FA) and Ψ the diagonal matrix. This diagonal matrix is the representation of uniqueness (total variance - communalities).  

2. In PCA, because of the absence of Ψ, we're not looking to determine a causal statement about the relationships. In FA, we're seek latent causes that will eventually be interpreted as the reason for our observed, measured variables. This is why in FA, we try to interpret and categorize the Factors. In PCA, not so much. We're simply finding a linear combination of our variables that create the 'Component', so there's no guarantee the Components are interpretable.

3. In FA, there are a lot of possible algorithms to choose from (depending on different assumptions). This can result in some algorithms resulting in a dead end (Heywood Case) and is a more of an art form than PCA. In PCA, just perform eigen decomposition and you're done.

It is worth noting that as the dimensionality (n) of our dataset increases, the differences between PCA and FA decreases. However for a small <i>'n'</i>, the difference is significant (Carine, 2014).


The first necessary step in PCA is to scale our data. If this isn't done first, then we'll have trouble when dealing with variances.

In [ ]:
#scaling our data similar to when doing FA:
df_clean_scaled = sc.fit_transform(df_clean)

In [ ]:
#converting it back into a panda:
df_clean_scaled = pd.DataFrame(df_clean_scaled, columns = list(df_clean.columns))
df_clean_scaled.head()

Before we decide how many PCA components, we need to know the criteria for how many components we should maintain. The cumulative variance to preserve our data seems to vary, but we'll choose the amount of components where 70-90% of the total variance is explained (Siswadi, Muslim & Bakhtiar, 2016). We'll explain in the discussion at the end of this of the alternative methods that determine how many PCA components to choose.

In [ ]:
#computing the eigenvalues & eigenvectors and seeing how many will fall between the 70-90% variance:
eig_vals, eig_vecs = np.linalg.eig(df_clean_scaled.corr())
count_pca_floor = float('inf')
count_pca_ceiling = 0
total_pca = sum(eig_vals)
total_pca_running = 0
i = 0
eig_vals = np.sort(eig_vals)[::-1]
print(eig_vals)
while total_pca_running <= 0.90:
    if total_pca_running >= 0.70 and count_pca_floor == float('inf'):
        count_pca_floor = min(count_pca_floor,i)
    if total_pca_running + eig_vals[i]/total_pca > 0.90:
        break
    total_pca_running += eig_vals[i]/total_pca
    count_pca_ceiling += 1
    i += 1
#the set calculated tells us the range of components we should keep to maximise variance but also minimise our dimensions
(count_pca_floor,count_pca_ceiling)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(df_clean_scaled)
pca_data = pca.transform(df_clean_scaled)
percent_var = np.round(pca.explained_variance_ratio_.cumsum()*100, decimals=1)
labels = ['PC' + str(x) for x in range(1, len(percent_var)+1)]
plt.figure(figsize=(20,20))
#cumulative variance explained:
plt.bar(x=range(1, len(percent_var)+1), height = percent_var, tick_label = labels, color='midnightblue', edgecolor='aqua', label = 'cumulative variance explained')
sns.lineplot(x=range(1, len(percent_var)+1), y = percent_var)
#benchmark of 70-90% variance explained:
plt.axhline(70.0, color = 'orange', linestyle="--")
plt.axhline(90.0, color = 'orange', linestyle="--")
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Cumulative Sum of Variance')
plt.show()

The cumulative variance explained shows there is evidence to keep at least 9 Principal Components. Becuase of this high number, we'll be unable to utilise one of PCA's advantages: its visualisations (best used for components under 4 since the human eye sees in 3D).

    We're going to start looking at the eigenvalues in the first few compoments and start to identify the features that correspond with the Components:

<b>Component 1:</b>

The output below suggests Component 1 is about Gold and Experience Difference


In [ ]:
aa = pd.DataFrame({'feature':df_clean_scaled.columns, 'eigenvalue': abs(pca.components_[0])})
aa.sort_values(by='eigenvalue', ascending=False).head(10)

<b>Component 2:</b>

The output below suggests Component 2 is about Kills/Deaths/Assists (K/D/A)

In [ ]:
aa = pd.DataFrame({'feature':df_clean_scaled.columns, 'eigenvalue': abs(pca.components_[1])})
aa.sort_values(by='eigenvalue', ascending=False).head(10)

<b>Component 3:</b>

The output below suggests Component 3 is about Elite Monster Kills. Interestingly, the eigenvalue for Herald kills is relatively small compared to Dragon kills, so we could conclude Component 3 is actually about Dragons taken instead.

In [ ]:
aa = pd.DataFrame({'feature':df_clean_scaled.columns, 'eigenvalue': abs(pca.components_[2])})
aa.sort_values(by='eigenvalue', ascending=False).head(10)

We can repeat the above steps to obtain the rest of the interpretations of our Components:

PC1: Gold/Exp Diff

PC2: K/D/A

PC3: Dragons Killed

PC4: Experience

PC5: Heralds Killed

PC6: Wards destroyed

PC7: Towers Destroyed

PC8: Wards Placed

PC9: Jungle Minions killed

PC10: (Eventually, the Compoments become less and less interpretable...)

With at least 70% of cumulative variance explained, the variance being < 1 after the 9th PC & the uninterpretability after the 9th PC and the 10th PC onwards having eigenvalues < 1, there is evidence to choose 9 Principal Components.

In [ ]:
#apply PCA to the number of above components:
pca9 = PCA(n_components=9)
pca9_fit = pca9.fit_transform(df_clean_scaled)
df_pca = pd.DataFrame(data = pca9_fit, columns = ['PCA' + str(i) for i in range(1,10)])
df_pca.head()

<b>Performing Logistic Regression after PCA:</b>

In [ ]:
px_train,px_test,py_train,py_test=train_test_split(pca9_fit,y,test_size=0.2,random_state=1)
PCA_LR=LogisticRegression() 

In [ ]:
PCA_LR.fit(px_train,py_train)

In [ ]:
result_PCALR=PCA_LR.predict(px_test)
print(accuracy_score(result_PCALR,py_test))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix_pca = confusion_matrix(py_test, result_PCALR)
print(confusion_matrix_pca)

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,pca9_fit)
result2=logit_model.fit()
print(result2.summary2())

<u><b>PCA retrospect and reconsiderations</b></u>

We've found a problem with our logistic regression - unsatisfactory p-values/z-scores in our Principal Components. There are clear signs of overemphasis on some contributions and underemphasis in others. The following attempts to explain why:

PCA is trying to achieve two things at the same time: Minimising the mean-squared error (MSE) of data points while also maximising the spread of variance. In other words - our resultant components should not only be an accurate interpretation/reconstruction of our variables (minimising MSE) but also construct new characteristics that strongly differ across variables (maximise variance). With that being said, let's pose two hypotheticals: What if our original dataset's variables had low colinearity? What if our original dataset's variables had high colinearity? (which our dataset has both).

If a dataset's features had little colinearity, PCA would be of almost no use to reduce dimensionality. If our dataset had 100 very unique and uncorrelated variables, then it would take 100 Principal Components to explain those 100 very unique and uncorrelated variables (Phan, 2016 pp. 6). This is why it's 'standard' practice to remove variables with low correlations, although this practice may be misleading and other approaches such as finding small subsets of original variables that approximate the various PCs in some optimal way (Silva, 2000). If a dataset's features had high colinearity, then the effect of adding more and more nearly correlated variables increases the contribution of the common underlying factor in PCA, causing PCA to overemphasise the contribution (Type2, 2013).

When analyzing how other individuals used PCA in this particular dataset in Kaggle, it seems like very little is addressed on these matters. Therefore, it may be preferable to search for small subsets of the original variables that approximate the relevant PCs in some optimal way (Silva, 2000). There are proposed algorithms and new criteria of what subsets of original variables will approximate the relevant PCs found in both Silva's <i>Discarding Variables in Principal Component Analysis: Algorithms for All-Subsets Comparisons</i> and Cadima & Jolliffe's <i>Variable selection and the interpretation of principal subspaces</i>.

However due to the computational intensity of either working out the criteria of each possible subset (2<sup>N</sup>-1 of non-empty subset possibilities), the ambiguity of choosing the specific subsets or the complexity of working out algorithms such as Yanai's GCD/McGabe's 4 Criterion from above, we won't touch this realm of PCA (at least in this notebook, maybe another one in the future?). Instead, we're going to narrow in to a much simpler method suggested by one of the same authors Jolliffe: the B2/B4 method (Joliffe, 2002, p. 138).



<u><b>PCA Selection methods</b></u>

The B2 procedure begins by doing PCA over our <i> n &#215; p</i> data matrix. If we choose to retain <i>q</i> Principal Components, then associate one variable with each of the last <i>p - q</i> variables. The <i>p - q</i> variables are then removed (Siswadi, Muslim & Bekhtiar, 2012). This can be done as a whole <i>p - q</i> block or one by one and performing multiple PCAs. Jolliffe explores the distinctive ways of deleting the variables and determines deleting en bloc consistently failed to select an appropriate subset for some simple correlation structures (Joliffe, 2002, p. 138).

The B4 procedure is associating one variable with each of the first <i>m</i> PCs, namely the variable not already chosen with the highest loading. This approach is complimentary to the B2 method, but also when there exists groups of highly correlated variables (which is almost all of our PCs) it is designed to select just one variable from each group.



In [ ]:
#Attempting first step of B2 method to delete the highest-loading variable of the least important PC:
aa = pd.DataFrame({'feature':df_clean_scaled.columns, 'eigenvalue': abs(pca.components_[22])})
aa.sort_values(by='eigenvalue', ascending=False).head(10)

Unfortunately, what we see is that the last eigenvalue has variables which have high loadings in our first (most variance explained) & last (least variance explained) PC, making the B2 method not very viable. 

Therefore, we'll first start by using the B4 method to assign variables to the first 9 PCs, then assign the remaining variables (for rejection) that did not appear in the B4 method:

<b>Highest eigenvalue of first 9 Components:</b>

In [ ]:
cols = []
for i in range(9):
    aa = pd.DataFrame({'feature':df_clean_scaled.columns, 'eigenvalue': abs(pca.components_[i])})
    x = aa.values.tolist()
    x.sort(key = lambda x: x[1], reverse = True)
    cols.append(x[0][0])
    print(x[0][0] + " : " + str(x[0][1]))
df_b4 = df_clean_scaled[cols]

<b>Performing Logistic Regression after B4 Method:</b>

In [ ]:
bx_train,bx_test,by_train,by_test=train_test_split(df_b4,y,test_size=0.2,random_state=1)
B4_LR=LogisticRegression()

In [ ]:
B4_LR.fit(bx_train,by_train)

In [ ]:
result_B4LR=B4_LR.predict(bx_test)
print(accuracy_score(result_B4LR,by_test))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix_b4 = confusion_matrix(by_test, result_B4LR)
print(confusion_matrix_b4)

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,df_b4)
result3=logit_model.fit()
print(result3.summary2())

Some of our p-values/z-scores are still unsatisfactory but much better than our initial PCA regression model.

<u><b> FA vs PCA Discussion:</b></u>

PCA compared to FA seems to be much more rigorously studied - with other researchers disputing the variance cutoff of our Kaiser Criterion (while we based our selections on λ<sub>x</sub> < 1, other researchers suggest cutoffs of 0.6-0.7) as well as introducing other criteria such as the broken-stick distribution (King & Jackson, 1999). I did perform a comparison of our PCA's explained variance ratio vs the broken-stick distribution. However when comparing the two, only the first two variances/eigenvalues were greater than the first two values, suggesting we only retain two variables. While this may be of discussion, we ignored the results in this notebook as two PCs could only explain a maximum of around 33%. Furthermore, Bartkowiak with an empirical comparison shows us that most of the rules lead to similar decisions on how many components/variables to keep - except the broken stick rule (Jolliffe, 2002 p. 132).

Because FA considers communality (while PCA doesn't), it makes it easier to distinguish variables which we should inherently remove after one iteration. We can see that using logistic regression after the B4 method results in some variables with higher p-values (blueWardsDestroyed, blueWardsPlaced, redTotalJungleMinionsKilled) - variables FA has already analyzed and has deemed unfit to use. While there are ways around this in PCA by analyzing multiple criteria and subsets, it seems like FA has been overall more effective in our notebook.

<u><b>Conclusion:</b></u>

Gold accumulation through early kills/solid last-hitting as well as securing early dragon kills maximises your chances in winning your League of Legends games.

The first 10 minutes of almost 10000 diamond-ranked games were analyzed through Factor Analysis and Principal Component Analysis. While both have their unique pros & cons - Factor Analysis proved to be more effective at analysing our dataset for latent variables while still achieving dimensionality reduction.

<b><u>References:</u></b>

Beauducel, A. (2001) <i>Problems with parallel analysis in data sets with oblique simple structure</i>. Institute for Science Education

Bishop, C.M. (2006) <i>Pattern Recognition and Machine Learning</i>. pp. 584

Cadima, J.F.C.L & Jolliffe, I.T (1995) <i>Variable selection and the interpretation of principal subspaces</i>. - Journal of Agricultural Biological and Environmental Statistics

Carine (2014) <i>Is there any good reason to use PCA instead of EFA? Also, can PCA be a substitute for factor analysis?</i>  Stack Exchange - Stats

Centre for Academic Success (2017) <i>Advice on Exploratory Factor Analysis</i>. Birmingham City University pp.2

Ferrari, S. (2013) <i>From Generative to Conventional Play: MOBA and League of Legends</i>. Georgia Institute of Technology

Hadi, N.U., Abdullah, N. & Sentosa, I. (2016) <i>An Easy Approach to Exploratory Factor Analysis: Marketing Perspective</i>. Rome-Italy: Journal of Education and Social Research, MSCER Publishing

Hayton, J.C., Allen D.G. & Scarpello, V. (2004) <i>Factor Retention Decisions in Exploratory Factor Analysis: A Tutorial on Parallel Analysis</i>. Sage Publications

Hilsdorf, M. (2020) <i>Cronbach’s Alpha: Theory and Application in Python</i>. Towards data science

IBM (2014) <i>Factor Analysis Rotation</i>. SPSS Statistics - 23.0.0

Joliffe, I.T. (2002) <i>Principal Component Analysis, Second Edition</i>. Springer

King, J.R. & Jackson, D.A. (1999) <i>Variable Selection in Large Environmental Data Sets using Principal Components Analysis</i>. Environmetrics

Li, J., Yang, M. & Liu, B. (2018) <i>Factor Analysis</i>. jbhender - github

Myzziah (2021) <i>Horn's Parallel Analysis in Python: Am I doing it correctly?</i> - reddit

Princeton University (2020) <i>Manuals - mvfactor</i>. stata.com

Silva, A.P.D (2000) <i>Discarding Variables in Principal Component Analysis: Algorithms for All-Subsets Comparisons</i>. The Catholic University of Portugal

Siswadi, Muslim, A. & Bakhtiar, T. (2012) <i>Variable Selection Using Principal Component and Procrustes Analyses and its Application in Educational Data</i>. Journal of Asian Scientific Research

Type2 (2013) <i>Should one remove highly correlated variables before doing PCA?</i>  Stack Exchange - Stats